In [1]:
# Dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
import joblib

In [2]:
# Read the CSV
df = pd.read_csv("../cleaned_data/cleaned_house_crime_school.csv")
df

,Suburb,Address,Rooms,Type,Price,Date,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,CrimeRate,NearbySchools
0,Abbotsford,85 Turner St,2,h,1480000.0,2016-12-03,2.5,3067,1,1,202,0,0,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019,157,2
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,2016-02-04,2.5,3067,1,0,156,79,1900,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019,157,2
2,Abbotsford,5 Charles St,3,h,1465000.0,2017-03-04,2.5,3067,2,0,134,150,1900,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019,157,2
3,Abbotsford,40 Federation La,3,h,850000.0,2017-03-04,2.5,3067,2,1,94,0,0,Yarra City Council,-37.7969,144.9969,Northern Metropolitan,4019,157,2
4,Abbotsford,55a Park St,4,h,1600000.0,2016-06-04,2.5,3067,1,2,120,142,2014,Yarra City Council,-37.8072,144.9941,Northern Metropolitan,4019,157,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16035,Yarraville,78 Bayview Rd,3,h,1101000.0,2018-02-24,6.3,3013,1,0,288,0,0,Maribyrnong City Council,-37.8110,144.8852,Western Metropolitan,6543,136,5
16036,Yarraville,13 Burns St,4,h,1480000.0,2018-02-24,6.3,3013,1,3,593,0,0,Maribyrnong City Council,-37.8105,144.8847,Western Metropolitan,6543,136,5
16037,Yarraville,29A Murray St,2,h,888000.0,2018-02-24,6.3,3013,2,1,98,104,2018,Maribyrnong City Council,-37.8155,144.8883,Western Metropolitan,6543,136,5
16038,Yarraville,147A Severn St,2,t,705000.0,2018-02-24,6.3,3013,1,2,220,120,2000,Maribyrnong City Council,-37.8229,144.8786,Western Metropolitan,6543,136,5


In [3]:
df=df[df["YearBuilt"]!=0]
df=df[df["BuildingArea"]!=0]
df

,Suburb,Address,Rooms,Type,Price,Date,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,CrimeRate,NearbySchools
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,2016-02-04,2.5,3067,1,0,156,79,1900,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019,157,2
2,Abbotsford,5 Charles St,3,h,1465000.0,2017-03-04,2.5,3067,2,0,134,150,1900,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019,157,2
4,Abbotsford,55a Park St,4,h,1600000.0,2016-06-04,2.5,3067,1,2,120,142,2014,Yarra City Council,-37.8072,144.9941,Northern Metropolitan,4019,157,2
6,Abbotsford,124 Yarra St,3,h,1876000.0,2016-05-07,2.5,3067,2,0,245,210,1910,Yarra City Council,-37.8024,144.9993,Northern Metropolitan,4019,157,2
7,Abbotsford,98 Charles St,2,h,1636000.0,2016-10-08,2.5,3067,1,2,256,107,1890,Yarra City Council,-37.8060,144.9954,Northern Metropolitan,4019,157,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16032,Wollert,27 Birchmore Rd,3,h,500000.0,2018-02-24,25.5,3750,2,2,383,118,2016,Whittlesea City Council,-37.6194,145.0395,Northern Metropolitan,2940,107,3
16034,Wollert,35 Kingscote Wy,3,h,570000.0,2018-02-24,25.5,3750,2,2,404,158,2012,Whittlesea City Council,-37.6103,145.0339,Northern Metropolitan,2940,107,3
16037,Yarraville,29A Murray St,2,h,888000.0,2018-02-24,6.3,3013,2,1,98,104,2018,Maribyrnong City Council,-37.8155,144.8883,Western Metropolitan,6543,136,5
16038,Yarraville,147A Severn St,2,t,705000.0,2018-02-24,6.3,3013,1,2,220,120,2000,Maribyrnong City Council,-37.8229,144.8786,Western Metropolitan,6543,136,5


# Select features (columns)

In [4]:
# Set features to be used as X values.
X = df.drop(["Price",
             "Suburb",
             "Date",
             "Address",
             "Type",
             "CouncilArea",
             "Lattitude",
             "Longtitude",
             "Regionname",
              "Postcode"], axis = "columns")
y = df["Price"]
print(X.shape, y.shape)

(7965, 10) (7965,)


In [5]:
# GET housing stats
total_houses = len(df)
max_value = df["Price"].describe()["max"]
min_value = df["Price"].describe()["min"]
print(f"Total houses: {total_houses}")
print(f"Highest price: {max_value}")
print(f"Lowest price: {min_value}")

Total houses: 7965
Highest price: 9000000.0
Lowest price: 131000.0


# Split the data into test and train data using `train_test_split` with test size of 33%

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.33)
X_train

,Rooms,Distance,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Propertycount,CrimeRate,NearbySchools
12747,3,16.5,2,2,228,135,2017,2474,173,1
188,3,11.1,2,3,555,130,1960,5132,155,3
3188,4,11.2,3,2,412,254,1930,8801,240,6
11141,4,15.5,2,2,779,185,1980,6202,115,3
2429,4,13.0,1,2,630,130,1960,8870,396,10
...,...,...,...,...,...,...,...,...,...,...
10377,2,13.1,1,2,177,84,2013,5070,201,6
10751,4,12.1,2,2,538,144,1970,10175,494,9
1578,3,9.3,2,2,357,144,1935,5051,221,4
15222,4,16.1,2,4,920,151,1995,8524,362,11


In [7]:
# Scale the data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
classifier = LogisticRegression(solver = "newton-cg", multi_class = "auto")
classifier.fit(X_train_scaled, y_train)

training_score = classifier.score(X_train_scaled, y_train)
base_accuracy = classifier.score(X_test_scaled, y_test)

print(f"LogisticRegression training Data Score: {training_score}")
print(f"LogisticRegression testing Data Score: {base_accuracy}")

LogisticRegression training Data Score: 0.015742128935532233
LogisticRegression testing Data Score: 0.011411182959300114


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [9]:
# Create the GridSearchCV model
param_grid = {"C": [0.1, 1, 10],
              "max_iter": [1000, 5000, 10000]}
grid = GridSearchCV(classifier, param_grid, error_score = "raise", verbose = 3)

In [10]:
# Train the model with GridSearch
grid.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


C:\Users\franc\anaconda3\envs\pythonData\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\franc\anaconda3\envs\pythonData\lib\site-packages\sklearn\utils\optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(


[CV 1/5] END .............C=0.1, max_iter=1000;, score=0.011 total time=332.8min


KeyboardInterrupt: 

In [ ]:
print(f"Best grid params: {grid.best_params_}")
print(f"Best grid score: {grid.best_score_}")

# Train Tuned Model

In [ ]:
# Tuned parameters
C = grid.best_params_["C"]
max_iter = grid.best_params_["max_iter"]

# Tuned model
tuned_model = LogisticRegression(solver = "newton-cg",
                                 multi_class = "auto",
                                 C = C,
                                 max_iter = max_iter)
tuned_model.fit(X_train_scaled, y_train)

tuned_model_score = tuned_model.score(X_train_scaled, y_train)
tuned_accuracy = tuned_model.score(X_test_scaled, y_test)

print(f"Training Data Score: {tuned_model_score}")
print(f"Testing Data Score: {tuned_accuracy}")

In [ ]:
# Make predictions with the hypertuned model
predictions = tuned_model.predict(X_test)
classifications = y_test.unique().tolist()

prediction_actual = {"Actual": y_test,
                     "Prediction": predictions}

prediction_df = pd.DataFrame(prediction_actual)
prediction_df = prediction_df.set_index("Actual").reset_index()
prediction_df

In [ ]:
evaluations = {"": ["Base Model", "Tuned Model"],
               "Accuracy": [f"%s" % round(base_accuracy, 3), f"%s" % round(tuned_accuracy, 3)]}

evaluations_df = pd.DataFrame(evaluations)
evaluations_df = evaluations_df.set_index("")

evaluations_df.to_csv("../evaluations/logistic_regression_eval.csv")
evaluations_df

# Save the model

In [ ]:
# filename = "../models/random_forest.sav"
# joblib.dump(tuned_model, filename)